<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, accuracy_score
import torch
import transformers
from tqdm import notebook

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
df.shape

(159571, 2)

In [6]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Подготовим данные для векторизации.

Для этого проведём кодировку символов к Unicode,

проведём лемматизацию слов,

удалим пробелы, пунктуацию и стоп-слова.



In [7]:
corpus = df['text'].values.astype('U')

In [8]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [22]:
corpus_clear = [lemmatize(clear_text(corpus[i])) for i in range(len(corpus))]

In [23]:
stopwords = set(nltk_stopwords.words('english'))

Разделим данные на тестовую и тренировочную выборки.

In [24]:
X_train, X_test, y_train, y_test = train_test_split(corpus_clear, df['toxic'], test_size=0.2, random_state=12345)

Проведём векторизацию с помощью TfidfVectorizer, а также удалим стоп-слова

In [25]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

In [26]:
X_train_vec = tf_idf.fit_transform(X_train)

In [27]:
X_test_vec = tf_idf.transform(X_test)

In [28]:
print('Размер тренировочного датасета:',X_train_vec.shape)
print('Размер тренировочного датасета:',X_test_vec.shape)

Размер тренировочного датасета: (127656, 144110)
Размер тренировочного датасета: (31915, 144110)


Данные подготовленные к обучению

## Обучение

Найдём метрику accuracy для константной модели. Эта модель будет всегда предсказывать 0

In [29]:
base = pd.Series(data=np.zeros((len(y_test))), index=y_test.index, dtype='int16')
base_accuacy = accuracy_score(y_test, base)
print('Accuracy константной модели:',round(base_accuacy,2))

Accuracy константной модели: 0.9


In [30]:
%%time
parameters = {'C': np.linspace(10, 20, num = 11, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
clf = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(X_train_vec, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV] C=10.0, max_iter=1000 ...........................................


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[CV] ............................ C=10.0, max_iter=1000, total=  16.1s
[CV] C=10.0, max_iter=1000 ...........................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   16.1s remaining:    0.0s


[CV] ............................ C=10.0, max_iter=1000, total=  16.3s
[CV] C=10.0, max_iter=1000 ...........................................
[CV] ............................ C=10.0, max_iter=1000, total=  15.8s
[CV] C=10.0, max_iter=1000 ...........................................
[CV] ............................ C=10.0, max_iter=1000, total=  15.9s
[CV] C=10.0, max_iter=1000 ...........................................
[CV] ............................ C=10.0, max_iter=1000, total=  16.8s
[CV] C=11.0, max_iter=1000 ...........................................
[CV] ............................ C=11.0, max_iter=1000, total=  15.8s
[CV] C=11.0, max_iter=1000 ...........................................
[CV] ............................ C=11.0, max_iter=1000, total=  15.8s
[CV] C=11.0, max_iter=1000 ...........................................
[CV] ............................ C=11.0, max_iter=1000, total=  15.5s
[CV] C=11.0, max_iter=1000 ...........................................
[CV] .

[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed: 15.5min finished


CPU times: user 9min 5s, sys: 6min 42s, total: 15min 47s
Wall time: 15min 49s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]),
                         'max_iter': [1000]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=2)

In [31]:
print('Наилучший показатель f1 на кросс-валидации:',clf.best_score_)
print('Параметр регуляризации для лучшей модели:',clf.best_params_)

Наилучший показатель f1 на кросс-валидации: 0.7617640513405083
Параметр регуляризации для лучшей модели: {'C': 16.0, 'max_iter': 1000}


## Тестирование 

In [32]:
model = LogisticRegression(C=16, max_iter=1000)
model.fit(X_train_vec, y_train)
predict = model.predict(X_test_vec)
f1 = f1_score(y_test, predict)
accuracy = accuracy_score(y_test, predict)

In [33]:
print('Показатель f1 на тестовой выборке:',round(f1,2))

Показатель f1 на тестовой выборке: 0.78


In [34]:
print('Показатель accuracy на тестовой выборке:',round(accuracy,2))

Показатель accuracy на тестовой выборке: 0.96


## Выводы

Данные были загружены и обработаны:
 * проведена лемматизация, удалены пробелы, знаки пунктуации и стоп-слова, корпус был векторизован.

На обновленных данных была обучена модель LogisticRegression:
 * показатель f1:0.78,
 * показатель accuracy:0.96, выше чем у константной модели,
 * кросс-валидация моделей и подбор гиперпараметров проводились с помощью GridSearchCV.